In [20]:
import numpy as np
import tensorflow as tf
import pandas as pd
import scipy
import os
from os import listdir
from scipy.io import loadmat
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split 
from tensorflow.keras import regularizers
from tensorflow.keras import activations

In [21]:

directory = "/home/work/HCOH/archive/seed_iv/eeg_feature_smooth/1/"
directories = ["/home/work/HCOH/archive/seed_iv/eeg_feature_smooth/{}/".format(i+1) for i in range(3)] 
print(directories)
channel_coords = [['0', '0', 'AF3', 'FP1', 'FPZ', 'FP2', 'AF4', '0', '0'], ['F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8'], ['FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8'], ['T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8'], ['TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8'], ['P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8'], ['0', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', '0'], ['0', '0', 'CB1', 'O1', 'OZ', 'O2', 'CB2', '0', '0']]

channel_list = ['FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', 'CB1', 'O1', 'OZ', 'O2', 'CB2']
print(len(channel_coords), len(channel_coords[0]))
coord_dict = {}
for n in range(len(channel_list)):
    for i, l in enumerate(channel_coords):
        for j, x in enumerate(l):
            if (channel_list[n] == x):
                coord_dict[n] = (i,j)
print(coord_dict)

['/home/work/HCOH/archive/seed_iv/eeg_feature_smooth/1/', '/home/work/HCOH/archive/seed_iv/eeg_feature_smooth/2/', '/home/work/HCOH/archive/seed_iv/eeg_feature_smooth/3/']
8 9
{0: (0, 3), 1: (0, 4), 2: (0, 5), 3: (0, 2), 4: (0, 6), 5: (1, 0), 6: (1, 1), 7: (1, 2), 8: (1, 3), 9: (1, 4), 10: (1, 5), 11: (1, 6), 12: (1, 7), 13: (1, 8), 14: (2, 0), 15: (2, 1), 16: (2, 2), 17: (2, 3), 18: (2, 4), 19: (2, 5), 20: (2, 6), 21: (2, 7), 22: (2, 8), 23: (3, 0), 24: (3, 1), 25: (3, 2), 26: (3, 3), 27: (3, 4), 28: (3, 5), 29: (3, 6), 30: (3, 7), 31: (3, 8), 32: (4, 0), 33: (4, 1), 34: (4, 2), 35: (4, 3), 36: (4, 4), 37: (4, 5), 38: (4, 6), 39: (4, 7), 40: (4, 8), 41: (5, 0), 42: (5, 1), 43: (5, 2), 44: (5, 3), 45: (5, 4), 46: (5, 5), 47: (5, 6), 48: (5, 7), 49: (5, 8), 50: (6, 1), 51: (6, 2), 52: (6, 3), 53: (6, 4), 54: (6, 5), 55: (6, 6), 56: (6, 7), 57: (7, 2), 58: (7, 3), 59: (7, 4), 60: (7, 5), 61: (7, 6)}


In [22]:
n = 24
perSample = ['de_movingAve', 'de_LDS', 'psd_movingAve', 'psd_LDS']
array = np.zeros(shape=(len(directories),len(os.listdir(directories[0])), n, 4, 8, 9, 5, 64)) # features = 4 datatypes*(8 x 9 eeg channel locs)*5 frequency bands*64 timestamps(zero padded) // trials = (3 sessions) x 15 people x 24 labels 
li = []
for h, dire in enumerate(directories):
    print(dire)
    data = [loadmat(dire + file) for file in os.listdir(dire)]
    for i, bigsample in enumerate(data):
        for j in range(n):
            for k, key in enumerate(perSample):
                sample = np.transpose(np.array(bigsample[key + str(j+1)]), (0,2,1))
                sample = np.pad(sample, [(0,0), (0,0), (0, 64-sample.shape[2])])
                for l, channel in enumerate(sample):
                    array[h][i][j][k][coord_dict[l][0]][coord_dict[l][1]] = channel

print(array.shape)

/home/work/HCOH/archive/seed_iv/eeg_feature_smooth/1/
/home/work/HCOH/archive/seed_iv/eeg_feature_smooth/2/
/home/work/HCOH/archive/seed_iv/eeg_feature_smooth/3/
(3, 15, 24, 4, 8, 9, 5, 64)


In [23]:
_X = array.reshape(np.prod(array.shape[0:3]), *array.shape[3:])
print(_X.shape)
X_loso = array[:,:,:,1,:,:,]
X_loso = np.transpose(X_loso, (0,1,2,6,3,4,5))
print(X_loso.shape)

(1080, 4, 8, 9, 5, 64)
(3, 15, 24, 64, 8, 9, 5)


In [24]:
session1_label = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3]
session2_label = [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1]
session3_label = [1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]
labels = {0: 'neutral', 1: 'sad', 2: 'fear', 3: 'happy'}

y = np.array(session1_label * 15 + session2_label * 15 + session3_label * 15)

print(y.shape)
y_loso = np.reshape(y, (3,15,24))
print(y_loso.shape)

(1080,)
(3, 15, 24)


In [36]:
X = _X.transpose(0, 5, 1,2,3,4)
print(X.shape)
X = X.reshape(X.shape[0], X.shape[1], np.prod(X.shape[2:]))
print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

(1080, 64, 4, 8, 9, 5)
(1080, 64, 1440)


In [37]:
X

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [38]:
y_train_onehot = pd.get_dummies(y_train).values
y_test_onehot = pd.get_dummies(y_test).values

In [39]:

X_train = X_train.astype('float32')
y_train_onehot = y_train_onehot.astype('float32')
X_test = X_test.astype('float32')
y_test_onehot = y_test_onehot.astype('float32')

In [40]:


from tensorflow.keras.layers import Conv1D , LSTM
from tensorflow.keras.layers import Input
from keras.layers import Dense, Flatten, Dropout
     

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, Flatten, MaxPooling1D
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
import gc

# Seed 설정
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# GPU 메모리 할당 방식 변경
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# 주어진 데이터
X = _X.transpose(0, 5, 1, 2, 3, 4)
X = X.reshape(X.shape[0], X.shape[1], np.prod(X.shape[2:]))
y = y

# 원-핫 인코딩
y_onehot = to_categorical(y)

# KFold 설정
kf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

# 결과 저장용 리스트
accuracy_list = []
f1_list = []
confusion_matrices = []

# KFold 교차 검증 수행
fold_no = 1
for train_index, test_index in kf.split(X, y):
    print(f'Training fold {fold_no}...')

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y_onehot[train_index], y_onehot[test_index]

    # Conv1D와 LSTM 모델 정의
    n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features)))  # 필터 수 감소
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(64))  # 유닛 수 감소
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))  # 유닛 수 감소
    model.add(Dense(n_outputs, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # 모델 훈련
    train_epochs = 30
    batch_size = 1  # 배치 크기 줄이기
    history = model.fit(X_train, y_train, epochs=train_epochs, batch_size=batch_size, verbose=True, validation_split=0.1)

    # 예측 및 평가
    predy = model.predict(X_test, batch_size=batch_size)
    predy = np.argmax(predy, axis=-1)
    y_test_labels = np.argmax(y_test, axis=-1)

    accuracy = accuracy_score(y_test_labels, predy)
    f1 = f1_score(y_test_labels, predy, average='weighted')
    cm = confusion_matrix(y_test_labels, predy)

    print(f'Fold {fold_no} - Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}')
    print(f'Confusion Matrix:\n{cm}')
    
    accuracy_list.append(accuracy)
    f1_list.append(f1)
    confusion_matrices.append(cm)

    fold_no += 1

    # GPU 메모리 초기화
    tf.keras.backend.clear_session()
    gc.collect()

# 최종 결과 출력
print('4-Fold Cross Validation Results:')
print(f'Accuracy: {np.mean(accuracy_list):.4f} ± {np.std(accuracy_list):.4f}')
print(f'F1 Score: {np.mean(f1_list):.4f} ± {np.std(f1_list):.4f}')

# 최종 혼동 행렬 출력
for i, cm in enumerate(confusion_matrices, 1):
    print(f'Confusion Matrix for Fold {i}:\n{cm}')

Training fold 1...


2024-07-15 16:16:14.391699: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1883] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 37742 MB memory:  -> device: 0, name: CUDA GPU, pci bus id: 0000:4c:00.0, compute capability: 8.0
2024-07-15 16:16:14.393991: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1883] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38284 MB memory:  -> device: 1, name: CUDA GPU, pci bus id: 0000:88:00.0, compute capability: 8.0


Epoch 1/30


2024-07-15 16:16:16.947370: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8907
2024-07-15 16:16:18.066697: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fe2638212c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-15 16:16:18.066749: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): CUDA GPU, Compute Capability 8.0
2024-07-15 16:16:18.066758: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): CUDA GPU, Compute Capability 8.0
2024-07-15 16:16:18.072451: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-15 16:16:18.156466: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


729/729 [==============================] - 7s 5ms/step - loss: 1.3859 - accuracy: 0.3032 - val_loss: 1.3524 - val_accuracy: 0.3086
Epoch 2/30
729/729 [==============================] - 3s 4ms/step - loss: 1.2703 - accuracy: 0.3992 - val_loss: 1.3400 - val_accuracy: 0.3457
Epoch 3/30
729/729 [==============================] - 3s 5ms/step - loss: 1.2120 - accuracy: 0.4184 - val_loss: 1.3676 - val_accuracy: 0.3457
Epoch 4/30
729/729 [==============================] - 3s 4ms/step - loss: 1.1898 - accuracy: 0.4184 - val_loss: 1.3316 - val_accuracy: 0.3457
Epoch 5/30
729/729 [==============================] - 3s 4ms/step - loss: 1.2012 - accuracy: 0.3964 - val_loss: 1.3304 - val_accuracy: 0.2222
Epoch 6/30
729/729 [==============================] - 3s 4ms/step - loss: 1.1855 - accuracy: 0.4060 - val_loss: 1.2475 - val_accuracy: 0.4321
Epoch 7/30
729/729 [==============================] - 3s 4ms/step - loss: 1.1449 - accuracy: 0.4417 - val_loss: 1.2297 - val_accuracy: 0.2716
Epoch 8/30
729/72

In [14]:
X = _X.transpose(0, 5, 1,2,3,4)
print(X.shape)
X = X.reshape(X.shape[0], X.shape[1], np.prod(X.shape[2:]))
print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

(1080, 64, 4, 8, 9, 5)
(1080, 64, 1440)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, Flatten, MaxPooling1D, Reshape
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.utils import to_categorical

# GPU 메모리 할당 방식 변경
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# 데이터 로드 및 전처리
# 데이터 로드 및 전처리
# X와 y 배열 준비
X = _X.transpose(0, 5, 1, 2, 3, 4)
X = X.reshape(X.shape[0], X.shape[1], np.prod(X.shape[2:]))
y = np.array(y)  # y는 라벨 데이터입니다.

# 원-핫 인코딩
y_onehot = to_categorical(y)

# KFold 설정
kf = KFold(n_splits=4, shuffle=True, random_state=42)

# 결과 저장용 리스트
accuracy_list = []
f1_list = []

# 모델 정의 함수들
def create_cnn_model(input_shape, output_shape):
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.4),
        Dense(output_shape, activation='softmax')
    ])
    return model

def create_lstm_model(input_shape, output_shape):
    model = Sequential([
        LSTM(64, activation='tanh', input_shape=input_shape),
        Dropout(0.4),
        Dense(64, activation='relu'),
        Dropout(0.4),
        Dense(output_shape, activation='softmax')
    ])
    return model

def create_hybrid_model(input_shape, output_shape):
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
        Dropout(0.3),
        MaxPooling1D(pool_size=2),
        LSTM(units=64, activation='tanh', return_sequences=True),
        Dropout(0.3),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.4),
        Dense(output_shape, activation='softmax')
    ])
    return model


# KFold 교차 검증 수행
fold_no = 1
for train_index, test_index in kf.split(X):
    print(f'Training fold {fold_no}...')

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y_onehot[train_index], y_onehot[test_index]

    # Conv1D와 LSTM 모델 정의
    n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

    cnn_model = create_cnn_model((n_timesteps, n_features), n_outputs)
    lstm_model = create_lstm_model((n_timesteps, n_features), n_outputs)
    hybrid_model = create_hybrid_model((n_timesteps, n_features), n_outputs)

    # 모델 컴파일
    cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    hybrid_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # 모델 훈련
    train_epochs = 30
    batch_size = 4  # 배치 크기 줄이기
    cnn_model.fit(X_train, y_train, epochs=train_epochs, batch_size=batch_size, verbose=True, validation_split=0.1)
    lstm_model.fit(X_train, y_train, epochs=train_epochs, batch_size=batch_size, verbose=True, validation_split=0.1)
    hybrid_model.fit(X_train, y_train, epochs=train_epochs, batch_size=batch_size, verbose=True, validation_split=0.1)

    # 예측 및 평가
    predy_cnn = np.argmax(cnn_model.predict(X_test, batch_size=batch_size), axis=-1)
    predy_lstm = np.argmax(lstm_model.predict(X_test, batch_size=batch_size), axis=-1)
    predy_hybrid = np.argmax(hybrid_model.predict(X_test, batch_size=batch_size), axis=-1)

    y_test_labels = np.argmax(y_test, axis=-1)

    accuracy_cnn = accuracy_score(y_test_labels, predy_cnn)
    f1_cnn = f1_score(y_test_labels, predy_cnn, average='weighted')

    accuracy_lstm = accuracy_score(y_test_labels, predy_lstm)
    f1_lstm = f1_score(y_test_labels, predy_lstm, average='weighted')

    accuracy_hybrid = accuracy_score(y_test_labels, predy_hybrid)
    f1_hybrid = f1_score(y_test_labels, predy_hybrid, average='weighted')

    print(f'Fold {fold_no} - CNN Accuracy: {accuracy_cnn:.4f}, CNN F1 Score: {f1_cnn:.4f}')
    print(f'Fold {fold_no} - LSTM Accuracy: {accuracy_lstm:.4f}, LSTM F1 Score: {f1_lstm:.4f}')
    print(f'Fold {fold_no} - Hybrid Accuracy: {accuracy_hybrid:.4f}, Hybrid F1 Score: {f1_hybrid:.4f}')

    accuracy_list.append((accuracy_cnn, accuracy_lstm, accuracy_hybrid))
    f1_list.append((f1_cnn, f1_lstm, f1_hybrid))

    fold_no += 1

    # GPU 메모리 초기화
    tf.keras.backend.clear_session()
    gc.collect()

# 최종 결과 출력
print('4-Fold Cross Validation Results:')
accuracy_array = np.array(accuracy_list)
f1_array = np.array(f1_list)
print(f'CNN Accuracy: {np.mean(accuracy_array[:,0]):.4f} ± {np.std(accuracy_array[:,0]):.4f}')
print(f'LSTM Accuracy: {np.mean(accuracy_array[:,1]):.4f} ± {np.std(accuracy_array[:,1]):.4f}')
print(f'Hybrid Accuracy: {np.mean(accuracy_array[:,2]):.4f} ± {np.std(accuracy_array[:,2]):.4f}')
print(f'CNN F1 Score: {np.mean(f1_array[:,0]):.4f} ± {np.std(f1_array[:,0]):.4f}')
print(f'LSTM F1 Score: {np.mean(f1_array[:,1]):.4f} ± {np.std(f1_array[:,1]):.4f}')
print(f'Hybrid F1 Score: {np.mean(f1_array[:,2]):.4f} ± {np.std(f1_array[:,2]):.4f}')

Training fold 1...
Epoch 1/30
183/183 [==============================] - 2s 5ms/step - loss: 242862848.0000 - accuracy: 0.2798 - val_loss: 133995.3750 - val_accuracy: 0.2469
Epoch 2/30
183/183 [==============================] - 1s 3ms/step - loss: 167893568.0000 - accuracy: 0.2894 - val_loss: 332945.4688 - val_accuracy: 0.2963
Epoch 3/30
183/183 [==============================] - 1s 3ms/step - loss: 268123888.0000 - accuracy: 0.3224 - val_loss: 665852.5625 - val_accuracy: 0.2963
Epoch 4/30
183/183 [==============================] - 1s 3ms/step - loss: 142387888.0000 - accuracy: 0.3141 - val_loss: 637777.7500 - val_accuracy: 0.3827
Epoch 5/30
183/183 [==============================] - 1s 3ms/step - loss: 21917326.0000 - accuracy: 0.3388 - val_loss: 331172.8438 - val_accuracy: 0.3704
Epoch 6/30
183/183 [==============================] - 1s 3ms/step - loss: 13700794.0000 - accuracy: 0.3416 - val_loss: 701762.6875 - val_accuracy: 0.3580
Epoch 7/30
183/183 [==============================] -